In [1]:
import glob
import pandas as pd
import numpy as np


In [2]:
frames = []
for path in glob.glob("../dataset/*.csv"):
    df = pd.read_csv(path, delimiter=";")
    #print(path)
    #print(df.head(2))
    frames.append(df)
data = pd.concat(frames)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,1,2,3,4,5,7,9,10,11,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.drop('BETRIEBSTAG', axis=1, inplace=True)
data.dropna(axis=0, subset=['PRODUKT_ID'], inplace=True)
data.drop('PRODUKT_ID', axis=1, inplace=True)
data.drop('BETREIBER_ABK', axis=1, inplace=True)
data.drop('BETREIBER_NAME', axis=1, inplace=True)
data.drop('FAHRT_BEZEICHNER', axis=1, inplace=True)
data.drop('BETREIBER_ID', axis=1, inplace=True)
data.drop('UMLAUF_ID', axis=1, inplace=True)
data.drop('VERKEHRSMITTEL_TEXT', axis=1, inplace=True)
data.drop('FAELLT_AUS_TF', axis=1, inplace=True)
data.drop('BPUIC', axis=1, inplace=True)
data.drop('ZUSATZFAHRT_TF', axis=1, inplace=True)
data.dropna(axis=0, inplace=True)

In [4]:
data.shape

(2692052, 10)

In [5]:
data.head(5)

,LINIEN_ID,LINIEN_TEXT,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
17,10057.0,IR,Thun,18.08.2016 13:48,18.08.2016 13:48,GESCHAETZT,18.08.2016 13:49,18.08.2016 13:49,GESCHAETZT,False
18,10057.0,IR,Spiez,18.08.2016 13:59,18.08.2016 13:57,GESCHAETZT,18.08.2016 14:00,18.08.2016 14:00,GESCHAETZT,False
23,101.0,ICE,Wuppertal Hbf,18.08.2016 05:49,18.08.2016 05:52,PROGNOSE,18.08.2016 05:51,18.08.2016 05:54,PROGNOSE,False
24,101.0,ICE,Köln Hbf,18.08.2016 06:47,18.08.2016 06:46,PROGNOSE,18.08.2016 06:55,18.08.2016 06:55,PROGNOSE,False
25,101.0,ICE,Siegburg/Bonn,18.08.2016 07:08,18.08.2016 07:09,PROGNOSE,18.08.2016 07:10,18.08.2016 07:11,PROGNOSE,False


In [ ]:
arr = (pd.to_datetime(data['ANKUNFTSZEIT']).astype(np.int64) - pd.to_datetime(data['AN_PROGNOSE']).astype(np.int64))/1000000000

In [ ]:
data10 = data.head(100000)

In [ ]:
dep = (pd.to_datetime(data10['ABFAHRTSZEIT']).astype(np.int64) - pd.to_datetime(data10['AB_PROGNOSE']).astype(np.int64))/1000000000

In [ ]:
data10.drop('ANKUNFTSZEIT', axis=1, inplace=True)
data10.drop('AN_PROGNOSE', axis=1, inplace=True)
data10.drop('ABFAHRTSZEIT', axis=1, inplace=True)
data10.drop('AB_PROGNOSE', axis=1, inplace=True)
data10.drop('AN_PROGNOSE_STATUS', axis=1, inplace=True)
data10.drop('AB_PROGNOSE_STATUS', axis=1, inplace=True)
data10.drop('DURCHFAHRT_TF', axis=1, inplace=True)

In [ ]:
data10.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
enc = OneHotEncoder()
lbl_text = LabelEncoder()

In [ ]:
lbl_text.fit(data10['LINIEN_TEXT'])

In [ ]:
lbl_text.transform(data10['LINIEN_TEXT'])

In [ ]:
enc.fit(lbl_text.transform(data10['LINIEN_TEXT']).reshape(-1,1))

In [ ]:
enc.transform(lbl_text.transform(data10['LINIEN_TEXT']).reshape(-1,1)).toarray()

In [ ]:
LINIEN_TEXT = pd.DataFrame(enc.transform(lbl_text.transform(data10['LINIEN_TEXT']).reshape(-1,1)).toarray())

In [ ]:
enc1 = OneHotEncoder()
lbl_text1 = LabelEncoder()

In [ ]:
lbl_text1.fit(data10['HALTESTELLEN_NAME'])

In [ ]:
enc1.fit(lbl_text1.transform(data10['HALTESTELLEN_NAME']).reshape(-1,1))

In [ ]:
enc1.transform(lbl_text1.transform(data10['HALTESTELLEN_NAME']).reshape(-1,1)).toarray()

In [ ]:
HALTESTELLEN_NAME = pd.DataFrame(enc1.transform(lbl_text1.transform(data10['HALTESTELLEN_NAME']).reshape(-1,1)).toarray())

In [ ]:
data10.drop('HALTESTELLEN_NAME', axis=1, inplace=True)
data10.drop('LINIEN_TEXT', axis=1, inplace=True)

In [ ]:
data10.index = range(0,len(data10))
X = pd.concat([data10, LINIEN_TEXT, HALTESTELLEN_NAME], axis=1)

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
arrivalReg = SGDRegressor(loss="huber")

In [ ]:
arrivalReg.fit(X.head(80000), arr.head(80000))

In [ ]:
arrivalReg.predict(X.tail(20000))

In [ ]:
arr.head(20000)